# Script Machine Debugging Tool (Libbitcoin c++)

<hr style="border: 0.5px dashed #000;">

In [1]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include")
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("bitcoin","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System.
#include <bitcoin/bitcoin.hpp> 

// Libbitcoin-Script Machine Script.
#include "supporting_scripts/script_machine.cpp"

### 1. Previous transaction output (UTXO).

**Previous output script** (fetch by `txid` & `index`)

In [2]:
!bx fetch-tx d4e68f7e163de4c31c6d4d75a38daa015a968371c1a94e2a182e29002235a168 -f json | jq -r  ".transaction.outputs[0].script" | bx script-encode


76a914b2b64b8f1c90e704d5893a08d7ee0471607c899788ac


In [3]:
auto previous_output_script_data = bc::to_chunk(
  bc::base16_literal("76a914b2b64b8f1c90e704d5893a08d7ee0471607c899788ac"));
bc::chain::script previous_output_script;
previous_output_script.from_data(previous_output_script_data, false);


**Previous output amount** (fetch by `txid` & `index`)

In [4]:
!bx fetch-tx d4e68f7e163de4c31c6d4d75a38daa015a968371c1a94e2a182e29002235a168 -f json | jq -r  ".transaction.outputs[0].value"


1964541


In [5]:
uint64_t previous_output_amount = 1964541;


### 2. Spending transaction.

In [6]:
// Paste your raw transaction data.
auto spending_transaction_data = bc::to_chunk(
  bc::base16_literal("010000000268a1352200292e182a4ea9c17183965a01aa8da3754d6d1cc3e43d167e8fe6d4000000006b483045022100ab52c6985a2a36eff74dca68f53cc5a7953ba5c3af901fe50dc3419c38f83da602204ccea2c8feca80a9eb17ed656f3e9431a4db177b2ce35540cc7a6c3802d917b583210396fc66cd2ce2254e6ff7466c64cb461c4896b5fc12b5f800ec682a8ac09514a0ffffffff68a1352200292e182a4ea9c17183965a01aa8da3754d6d1cc3e43d167e8fe6d4010000006a473044022005e7eca2047609576ce0d1993ea2bead42f87d8c85ffdf8cc6841c5e8834eea80220659711e8a4095648c1c3527c7f884ac1ed7c0c9837b10f4f18e70d815e383357832102db969196e81611b69ef3bd15a86588dc24e5c381a1e4c9f614d08b6dfbcd6293ffffffff0289f41d00000000001976a914736b4005d4999db71c7184f7c716c43d6f89685488ac89f41d00000000001976a91493ecb4f92c41105d8a2e0937c3dac5860e751eca88ac00000000"));
bc::chain::transaction spending_transaction;
auto tx_set = spending_transaction.from_data(spending_transaction_data, true, false); // wire=true, witness=false


**Index of spending input.**

In [7]:
uint32_t input_index = 0;

### 3. Evaluate input, output, p2sh and witness script runs.

Run following cell to evaluate the `previous output` | `input` script(s).

In [8]:
bc::code ec = evaluate_in_out_p2sh_witness_runs(previous_output_script, previous_output_amount,
  spending_transaction, input_index, bc::machine::rule_fork::all_rules);
std::cout << ec.message() << std::endl;


=========== Input script evaluation ===========

> Operation: 0
[3045022100ab52c6985a2a36eff74dca68f53cc5a7953ba5c3af901fe50dc3419c38f83da602204ccea2c8feca80a9eb17ed656f3e9431a4db177b2ce35540cc7a6c3802d917b583]

> Stack after operation: 0
[3045022100ab52c6985a2a36eff74dca68f53cc5a7953ba5c3af901fe50dc3419c38f83da602204ccea2c8feca80a9eb17ed656f3e9431a4db177b2ce35540cc7a6c3802d917b583]

>> Operation: 1
[0396fc66cd2ce2254e6ff7466c64cb461c4896b5fc12b5f800ec682a8ac09514a0]

>> Stack after operation: 1
[0396fc66cd2ce2254e6ff7466c64cb461c4896b5fc12b5f800ec682a8ac09514a0]
[3045022100ab52c6985a2a36eff74dca68f53cc5a7953ba5c3af901fe50dc3419c38f83da602204ccea2c8feca80a9eb17ed656f3e9431a4db177b2ce35540cc7a6c3802d917b583]

=========== Output script evaluation ==========

> Operation: 0
dup

> Stack after operation: 0
[0396fc66cd2ce2254e6ff7466c64cb461c4896b5fc12b5f800ec682a8ac09514a0]
[0396fc66cd2ce2254e6ff7466c64cb461c4896b5fc12b5f800ec682a8ac09514a0]
[3045022100ab52c6985a2a36eff74dca68f53cc5a7953ba